In [70]:
path=r'C:/Users/lenovo/Desktop/ML Project/dataset/'

In [71]:
import pandas as pd
import numpy as np
train = pd.read_csv("../dataset/train.csv")
X = pd.read_csv("../embeddings/use_embeddings.csv")
test = pd.read_csv("../embeddings/use_embeddings_test.csv")
labels = pd.read_csv(path+"test_labels.csv")

In [5]:
#X.head()
classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[classes].values
y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

In [6]:
input_size = 512
hidden_size = 25
num_labels = 6

In [28]:
def sigmoid(z):
    
    #ReLU:
    #return np.maximum(x, 0)
    #x = (np.abs(x) + x) / 2.0

    #return x
    return 1 / (1 + np.exp(-z))


In [29]:
def forward_propagate(X, theta1, theta2):
    m = X.shape[0]
    
    a1 = np.insert(X, 0, values=np.ones(m), axis=1)
    z2 = a1 * theta1.T
    a2 = np.insert(sigmoid(z2), 0, values=np.ones(m), axis=1) #insert new bias
    z3 = a2 * theta2.T
    h = sigmoid(z3)

    return a1, z2, a2, z3, h

In [30]:
def cost(theta1, theta2, X, y):
    m = len(X)
    X = np.matrix(X)
    y = np.matrix(y)
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    print(h)
    
    # compute the cost
    J=0
    J= -np.sum(np.multiply(y,np.log(h))+ np .multiply((1-y),np.log(1-h)))
    return J / m

In [31]:
# the theta parameter vector 
# He initialization
#params = np.random.random(size=hidden_size * (input_size + 1) + num_labels * (hidden_size + 1))  * np.sqrt(2 / input_size)
params = (np.random.random(size=hidden_size * (input_size + 1) + num_labels * (hidden_size + 1)) - 0.5) * 0.25

# the theta matrix derived from params
theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
theta1.shape,theta2.shape

((25, 513), (6, 26))

In [32]:
cost(theta1, theta2, X, y)

[[0.4935239  0.53611841 0.45718205 0.53970042 0.46454421 0.44707005]
 [0.49028503 0.5361459  0.45924822 0.53889992 0.46363422 0.44433474]
 [0.49090845 0.53599394 0.45801973 0.53916533 0.46554205 0.44812923]
 ...
 [0.49230295 0.53490537 0.45693191 0.53842232 0.46667397 0.44940176]
 [0.49090218 0.53538539 0.45937098 0.53802836 0.46452254 0.44672405]
 [0.49324373 0.53590055 0.45861504 0.53857715 0.46503444 0.44734653]]


4.072015452206742

In [33]:
def sigmoid_gradient(z):    # derivative of sigmoid 
    
    #ReLU
    #x[x <= 0] = 0
    #x[x > 0] = 1

    #return x
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))

In [34]:
def backprop(params,  X, y):
    m = len(X)
    X = np.matrix(X)
    y = np.matrix(y)
    
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # initializations
    J = 0
    delta1 = np.zeros(theta1.shape)# (25, 401)
    delta2 = np.zeros(theta2.shape)# (10, 26)
    
    # compute the cost
    for i in range(m):
        first_part =np.multiply(-y[i,:],np.log(h[i,:]))
        second_part =np.multiply((1-y[i,:]),np.log(1-h[i,:]))
        J+=np.sum(first_part-second_part)
    J = J/m
    
    # perform backpropagation
    for t in range(m):#m=5000
        
        a_1t = a1[t,:]
        z_2t=z2[t,:]
        z_2t=np.insert(sigmoid_gradient(z_2t),0,values=np.ones(1))
        a_2t = a2[t,:]  
        z_3t=z3[t,:]
        ht=h[t,:]#(1,10)
        yt = y[t,:]#(1,10)
        d3_t=ht-yt
            
        d2_t=np.multiply((theta2.T*d3_t.T).T,z_2t)#(1,26)
        delta1=delta1+np.multiply(d2_t[:,1:].T,a_1t)
        delta2=delta2+np.multiply(d3_t.T,a_2t)

    delta1 = delta1 / m
    delta2 = delta2 / m

    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))
    return J,grad

In [35]:
J, grad = backprop(params, X, y)
J, grad.shape

(4.072015452206786, (12981,))

In [37]:
from scipy.optimize import minimize
#params = np.concatenate((np.ravel(theta1), np.ravel(theta2)))
# minimize the objective function
fmin = minimize(fun=backprop, x0=params, args=(X, y), 
                method='TNC', jac=True, options={'maxiter': 100})
fmin

     fun: 0.35781605324886867
     jac: array([-4.01852853e-04,  6.41655214e-06,  3.26822965e-06, ...,
        5.97044596e-04,  4.64399673e-04,  1.05502046e-03])
 message: 'Max. number of function evaluations reached'
    nfev: 101
     nit: 12
  status: 3
 success: False
       x: array([-0.38216075,  0.05357219,  0.27107219, ..., -0.65317092,
       -0.37540756, -1.25353737])

In [73]:
def predict(X):
    X = np.matrix(X)
    #print(X.shape)
    theta1 = np.matrix(np.reshape(fmin.x[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(fmin.x[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    #print(theta1)
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    return h

In [67]:
from sklearn.metrics import roc_auc_score

In [74]:
# AUC of the training set:
y_pred = predict(X)
train_roc_auc = roc_auc_score(y, y_pred)
print('Train ROC AUC Score:', train_roc_auc)

D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:590: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


Train ROC AUC Score: 0.969262031040528


In [75]:
# AUC of the test set:
labels = labels[classes]
sum_labels=np.sum(labels.values,axis=1)
idx=sum_labels>=0
y_test = labels[idx]
X_test = test[idx]

y_pred_test = predict(X_test)
test_roc_auc = roc_auc_score(y_test, y_pred_test)
print('Test ROC AUC Score:', test_roc_auc)

Test ROC AUC Score: 0.9586175878003429


D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:590: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
